Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
pip install yfinance

     |████████████████████████████████| 6.3 MB 7.7 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=0999468d9dc58b65948c6db530a7ada98e4357e1e8a4bb92540b79498d7e9ee6
  Stored in directory: /root/.cache/pip/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
pip install pandas_datareader

In [4]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import yfinance as yf

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [5]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = "TSLA"

In [6]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo(stock_symbol,pd.Timestamp('2019'),pd.Timestamp('2021'))
df = df[df.columns[0:4]]
df = df[['Open', 'High', 'Low', 'Close']]
df = df.dropna()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2019-01-02,61.220001,63.026001,59.759998,62.023998
2019-01-03,61.400002,61.880001,59.476002,60.071999
2019-01-04,61.200001,63.599998,60.546001,63.537998
2019-01-07,64.344002,67.348000,63.549999,66.991997
2019-01-08,68.391998,68.802002,65.403999,67.070000
...,...,...,...,...
2020-12-24,642.989990,666.090027,641.000000,661.770020
2020-12-28,674.510010,681.400024,660.799988,663.690002
2020-12-29,661.000000,669.900024,655.000000,665.989990


In [7]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df['Close'].rolling(window=15).mean()
df['Open_1_Change'] = df['Open'].diff()
df = df.dropna()
X = df

In [8]:
X.isna().any()

Open                False
High                False
Low                 False
Close               False
Close_15_Rolling    False
Open_1_Change       False
dtype: bool

In [9]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where((df.Close.shift(-1)-df['Close'])>=0,1,-1)

In [10]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train = X[:index]
X_test = X[index:]
y_train = y[:index]
y_test = y[index:]

In [11]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train,y_train)

In [12]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame({'column':X.columns.values, 'b': logistic.coef_.ravel()})

,column,b
0,Open,-0.037933
1,High,0.019647
2,Low,0.083481
3,Close,-0.059696
4,Close_15_Rolling,-0.003120
5,Open_1_Change,0.028716


In [13]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)

In [14]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

metrics.confusion_matrix(y_pred=predictions,y_true=y_test)

array([[16, 45],
       [25, 62]])

In [15]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

logistic.score(X_test,y_test)

0.527027027027027

In [18]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val = cross_val_score(LogisticRegression(),X=X,y=y,cv=5)
cross_val

array([0.55555556, 0.54081633, 0.48979592, 0.5       , 0.57142857])